In [ ]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

Installed package MathNet.Numerics.FSharp version 4.15.0

In [ ]:
open MathNet.Numerics.LinearAlgebra
open MathNet.Numerics.Integration
open System.Collections.Generic

Рассмотрим краевую задачу для линейного обыкновенного дифференциального уравнения
второго порядка с однородными граничными условиями.

In [ ]:
let l u du d2u = fun x ->
    - (6. + x) / (7. + 3. * x) * d2u x
    - (1. - x / 2.) * du x
    + (1. + 0.5 * cos x) * u x

let f = fun x -> 1. - x / 3.

let left = -1.
let right = 1.

let (alpha1, alpha2) = -2., -1
let (beta1, beta2) = 0., 1

In [ ]:
let k = 2

/// k - верхний индекс, n - максимальная степень
let jacobiPolynomials k =
    let rec recurrenceRelation n prevPoly2 prevPoly = seq {
        yield prevPoly2
        yield! 
            fun (x: float) -> 
                (
                    (float n + float k + 2.) * (2. * float n + 2. * float k + 3.) * x * prevPoly x - 
                    (float n + float k + 2.) * (float n + float k + 1.) * prevPoly2 x
                ) 
                / ((float n + 2. * float k + 2.) * (float n + 2.))
            |> recurrenceRelation (n + 1) prevPoly
    }   

    recurrenceRelation 0 (fun x -> 1.) (fun x -> (float k + 1.) * x) 

let jacobiDerivatives k =
    let jacobi = jacobiPolynomials (k + 1)

    let rec recurrenceRelation n = seq {
        yield fun (x: float) -> (float n + 2. * float k + 1.) / 2. * (jacobi |> Seq.item (n - 1)) x
        yield! recurrenceRelation (n + 1)
    }   

    seq {
        yield (fun x -> 0.) 
        yield! recurrenceRelation 1
    }
   
let jacobiDerivatives2 k = 
    let jacobi = jacobiPolynomials (k + 2)

    let rec recurrenceRelation n = seq {
        yield fun (x: float) -> (float n + 2. * float k + 1.) * (float n + 2. * float k + 2.) / 4. * (jacobi |> Seq.item (n - 2)) x
        yield! recurrenceRelation (n + 1)
    }   

    seq {
        yield (fun x -> 0.) 
        yield (fun x -> 0.) 
        yield! recurrenceRelation 2  
    } 

In [ ]:
let w1 = fun x -> x ** 2. - 2. * x - 5.
let dw1 = fun x -> 2. * x - 2.
let ddw1 = fun x -> 2.

let w2 = fun x -> x ** 3. - 3. * x + 2.
let dw2 = fun x -> 3. * x ** 2. - 3.
let ddw2 = fun x -> 6. * x

In [ ]:
let coordinateFunctionsAndDerivatives =
    let jacobi = jacobiPolynomials k
    let jacobiD = jacobiDerivatives k
    let jacobiD2 = jacobiDerivatives2 k

    let rec recurrenceRelation n = seq {
        yield 
            (fun (x: float) -> (x ** 2. - 1.) ** 2. * (Seq.item (n - 2) jacobi) x),
            (fun (x: float) -> 4. * (x ** 2. - 1.) * x * (Seq.item (n - 2) jacobi) x + (x ** 2. - 1.) ** 2. * (Seq.item (n - 2) jacobiD) x),
            (fun (x: float) -> 4. * (3. * x ** 2. - 1.) * (Seq.item (n - 2) jacobi) x + 8. * (x ** 2. - 1.) * x * (Seq.item (n - 2) jacobiD) x + (x ** 2. - 1.) ** 2. * (Seq.item (n - 2) jacobiD2) x)
            
        yield! recurrenceRelation (n + 1)
    } 

    seq {
        yield (w1, dw1, ddw1)
        yield (w2, dw2, ddw2)
        yield! recurrenceRelation 2
    }

In [ ]:
let n = 7
let coordinates = 
    coordinateFunctionsAndDerivatives 
    |> Seq.take n
    |> Seq.toList

let a = DenseMatrix.init n n (fun i j ->
    let (wi, dwi, ddwi) = coordinates.[i]
    let (wj, dwj, ddwj) = coordinates.[j]
    let lwj = l wj dwj ddwj
    let lwi = l wi dwi ddwi
    GaussLegendreRule.Integrate((fun x -> lwj x * lwi x), left, right, 20)
)    

let b = DenseVector.init n (fun i ->
    let (wi, dwi, ddwi) = coordinates.[i]
    let lwi = l wi dwi ddwi
    GaussLegendreRule.Integrate((fun x -> f x * lwi x), left, right, 20)
)

type Info = {
    N: int
    CondA: float
    Yn_05: float
    Yn0: float
    Yn05: float
}

let table = List<Info>()

for k = 3 to n do
    let c = a.[0 .. k - 1, 0 .. k - 1].Solve(b.[0 .. k - 1])
    let u = fun x ->
        [ 0 .. k - 1 ]
        |> List.map 
            (fun i -> 
                let (wi, _, _) = coordinates.[i]
                wi x * c.[i]
            )
        |> List.sum
    
    {
        N = k
        CondA = a.[0 .. k - 1, 0 .. k - 1].ConditionNumber()
        Yn_05 = u -0.5
        Yn0 = u 0.
        Yn05 = u 0.5
    }
    |> table.Add

table

index,N,CondA,Yn_05,Yn0,Yn05
0,3,5.786255329148751,0.6098184266667431,0.6060126179916787,0.5848324252786177
1,4,8.094225087794854,0.608045316081597,0.6065175984394242,0.5859412122229799
2,5,23.580941551554865,0.6080405825536018,0.6065469070566012,0.5859221380212414
3,6,52.502377244340714,0.6080130399275563,0.6065717018130615,0.5859292790793892
4,7,99.59431356200206,0.6080022017688684,0.6065868464282662,0.5859149825869558


In [ ]:
let getChebyshevRoots n = 
    [0 .. (n - 1)]
    |> List.map (fun k -> Math.Cos (Math.PI * (float k + 0.5) / float n))

In [ ]:
let roots = getChebyshevRoots n

let aCollocation = DenseMatrix.init n n (fun i j ->
    let (wj, dwj, ddwj) = coordinates.[j]
    let lwj = l wj dwj ddwj
    lwj roots.[i]
)    

let bCollocation = DenseVector.init n (fun i ->
    f roots.[i]
)

let tableCollocation = List<Info>()

for k = 3 to n do
    let cCollocation = aCollocation.[0 .. k - 1, 0 .. k - 1].Solve(bCollocation.[0 .. k - 1])
    let uCollocation = fun x ->
        [ 0 .. k - 1 ]
        |> List.map 
            (fun i -> 
                let (wi, _, _) = coordinates.[i]
                wi x * cCollocation.[i]
            )
        |> List.sum
    
    {
        N = k
        CondA = aCollocation.[0 .. k - 1, 0 .. k - 1].ConditionNumber()
        Yn_05 = uCollocation -0.5
        Yn0 = uCollocation 0.
        Yn05 = uCollocation 0.5
    }
    |> tableCollocation.Add

tableCollocation

index,N,CondA,Yn_05,Yn0,Yn05
0,3,38.82904065986681,0.5340079713683559,0.5691320707841788,0.5667434587065273
1,4,60.74343360914534,0.5967014383014546,0.6017776928304563,0.5835177495540811
2,5,59.092582383775806,0.6091822108927245,0.6070555612013798,0.5861521546401531
3,6,14.378346482311823,0.6082091371572527,0.6066719995582612,0.5859587438793576
4,7,14.268023021426973,0.6080020227612992,0.6065877319676227,0.5859175623013835
